In [2]:
import numpy as np
import pandas as pd
import itertools

In [ ]:
encoding='latin1'

In [15]:
# these functions are to find the row index of desired data
def cut_line(line, cut_symbol = '|'):
    cut_list = [l for l in (line).split(cut_symbol)]
    if '' in cut_list:
        cut_list.remove('')
    if '\n' in cut_list:
        cut_list.remove('\n')
    return cut_list


def return_new_lines(file_path, key_1, key_2, find_length=False):
    with open(file_path, encoding='latin1') as f:
        lines = f.readlines()
    new_lines = [cut_line(line) for line in lines]

    for i in range(len(new_lines)):
        if key_1 in new_lines[i] and key_2 in new_lines[i]:
            start = i+2
            col = i
            break
            
    if find_length:
        lengths = [len(ele) for ele in new_lines[start]]
    else:
        lengths = 'KPMG'
    return new_lines, lengths, start, col


def store_index(file_path, key_1, key_2, key_3):
    new_lines, _, _, _ = return_new_lines(file_path, key_1, key_2, find_length=False)

    begin_index, endin_index = [], []
    for i in range(len(new_lines)):
        if key_1 in new_lines[i] and key_2 in new_lines[i]:
            begin_index.append(i+2)
        if key_3 not in new_lines[i] and key_3 in new_lines[i-1]: 
            endin_index.append(i)

    if key_2 == 'Eingangssteuer in Fremdw.     ':
        gap = len(endin_index) - len(begin_index)
        endin_index = [endin_index[z+gap] for z in range(len(begin_index))]

    save_index = []
    for j in range(len(begin_index)):
        temp_save_index = list(range(begin_index[j], endin_index[j]))
        save_index.append(temp_save_index)
    return save_index


In [25]:
# these functions are to separate row elements using lengths as the constraint and form dataframe
def read_lines(file_path):
    with open(file_path,encoding='latin1') as f:
        lines = f.readlines()
    return lines


def separate_row_element(lengths, line):
    row = []
    count = 1
    for i in range(len(lengths)):
        row_element = []
        for j in range(lengths[i]):
            row_element.append(line[count])
            count+=1
        count+=1
        row.append(''.join(row_element))
    return row


def generate_df(new_lines, col, start, save_index, lines):
    temp_list = []
    columns = new_lines[col] 

    for m in range(len(save_index)):
        for n in range(len(save_index[m])):
            if len(lines[save_index[m][n]]) == len(lines[start]):
                line = separate_row_element(lengths, lines[save_index[m][n]])
                temp_list.append(line)
            else: 
                print("wrong length error: should be: {}, ".format(len(lines[start])), "current: {}. " .format(len(lines[save_index[m][n]])), "row index: {}" .format(save_index[m][n]))
    if key_2 == 'Ausgangssteuer in Fremdw.     ':  
        print('for Ausgangssteuer, there are in total {} rows.'.format(len(temp_list)))
    else:
        print('for Eingangssteuer, there are in total {} rows.'.format(len(temp_list)))
    
    print('\n')
    df = pd.DataFrame(temp_list, columns=columns)
    df.head()
    return df

In [26]:
# this function is to export dataframe as desired format
def export_data(type, df, key_2): # excel or csv
    if type == 'excel':
        if key_2 == 'Ausgangssteuer in Fremdw.     ':
            df.to_excel("Ausgangssteuer.xlsx")
        else:
            df.to_excel("Eingangssteuer.xlsx")
    if type == 'csv':
        if key_2 == 'Ausgangssteuer in Fremdw.     ':
            df.to_csv("Ausgangssteuer.csv")
        else:
            df.to_csv("Eingangssteuer.csv")

In [66]:
# path of .txt
file_path = '/Users/rampageyao/Desktop/KPMG Work/KPMG/Tax_Cleaning/AKW 2020.TXT' 

# arbitrary symbol from desired classes
key_1 = 'BuKr' 

# options: 'Ausgangssteuer in Fremdw.     ' or 'Eingangssteuer in Fremdw.     '
key_2 = 'Ausgangssteuer in Fremdw.     ' 

# arbitrary symbol from desired data value
key_3 = 'DE10' 

In [71]:

save_index = store_index(file_path, key_1, key_2, key_3)
print(save_index[0])

[31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85]


In [83]:
new_lines, lengths, start, col = return_new_lines(file_path, key_1, key_2, find_length=True)
print(lengths, start)

[4, 2, 4, 10, 10, 10, 10, 3, 2, 3, 7, 23, 23, 5, 5, 8, 10, 20, 35, 3, 15, 35, 35, 10, 10, 12, 30, 16, 23, 10, 16, 30] 31


In [35]:
print(lengths)
print(len(lengths))

[4, 2, 4, 10, 10, 10, 10, 3, 2, 3, 7, 23, 23, 5, 5, 8, 10, 20, 35, 3, 15, 35, 35, 10, 10, 12, 30, 16, 23, 10, 16, 30]
32


In [23]:
lines = read_lines(file_path)

In [24]:
print(lines[31])

|DE10| 1|2020|01.01.2020|01.01.2020|          |1000007940|AB |A9|MWS|  0,000|                  0,01 |                       |EUR  |     |5521100 |1090100   |                    |TUDAWE TRADING COMPANY PVT LTD     |LK |Sri Lanka      |No. 509, Elvitigala Road           |Colombo                            |05        |01.01.2020|            |Ausgangssteuer Ausland steuer |           0,00 |                  0,01 |910373337 |           0,00 |                              |



In [45]:
print(lines[31][0])
print(lines[31][1])
print(lines[31][2])
print(len(lines[31]))

|
D
E
473


In [40]:
test_row = []
test_row = [ele for ele in lines[31]]
print(test_row)
print('\n')
print(''.join(test_row))

['|', 'D', 'E', '1', '0', '|', ' ', '1', '|', '2', '0', '2', '0', '|', '0', '1', '.', '0', '1', '.', '2', '0', '2', '0', '|', '0', '1', '.', '0', '1', '.', '2', '0', '2', '0', '|', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '|', '1', '0', '0', '0', '0', '0', '7', '9', '4', '0', '|', 'A', 'B', ' ', '|', 'A', '9', '|', 'M', 'W', 'S', '|', ' ', ' ', '0', ',', '0', '0', '0', '|', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '0', ',', '0', '1', ' ', '|', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '|', 'E', 'U', 'R', ' ', ' ', '|', ' ', ' ', ' ', ' ', ' ', '|', '5', '5', '2', '1', '1', '0', '0', ' ', '|', '1', '0', '9', '0', '1', '0', '0', ' ', ' ', ' ', '|', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '|', 'T', 'U', 'D', 'A', 'W', 'E', ' ', 'T', 'R', 'A', 'D', 'I', 'N', 'G', ' ', 'C', 'O', 'M', 'P', 'A', 'N', 'Y',

In [79]:
row = separate_row_element(lengths, lines[31])

In [80]:
print(row)

['DE10', ' 1', '2020', '01.01.2020', '01.01.2020', '          ', '1000007940', 'AB ', 'A9', 'MWS', '  0,000', '                  0,01 ', '                       ', 'EUR  ', '     ', '5521100 ', '1090100   ', '                    ', 'TUDAWE TRADING COMPANY PVT LTD     ', 'LK ', 'Sri Lanka      ', 'No. 509, Elvitigala Road           ', 'Colombo                            ', '05        ', '01.01.2020', '            ', 'Ausgangssteuer Ausland steuer ', '           0,00 ', '                  0,01 ', '910373337 ', '           0,00 ', '                              ']


In [84]:
print(new_lines[col] )

['BuKr', ' M', 'Jahr', 'Buch.dat. ', 'Belegdatum', 'Meldedatum', 'Belegnr   ', 'Art', 'St', 'Vor', 'Satz   ', '      Steuerbasisbetrag', ' Basisbetrag in Fremdw.', 'Währ ', 'FWähr', 'Steuerk.', 'Hauptbuch ', 'Umsatzsteuer-Id.Nr  ', 'Geschäftspartnername               ', 'ISO', 'Land           ', 'Straße                             ', 'Ort                                ', 'Postleitz.', 'Ausgleich ', '  UmrechKurs', 'Beschreibung                  ', 'abzuf. Ausg. St.', '           Bruttobetrag', 'Debitor   ', '  Ausgangssteuer', 'Ausgangssteuer in Fremdw.     ']


In [87]:
temp_list = []
columns = new_lines[col] 

for m in range(len(save_index)):
    for n in range(len(save_index[m])):
        line = separate_row_element(lengths, lines[save_index[m][n]])
        temp_list.append(line)
        #print(save_index[m][n])
if key_2 == 'Ausgangssteuer in Fremdw.     ':  
    print('for Ausgangssteuer, there are totoal {} rows.'.format(len(temp_list)))
else:
    print('for Eingangssteuer, there are totoal {} rows.'.format(len(temp_list)))

df = pd.DataFrame(temp_list, columns=columns)

IndexError: string index out of range

In [92]:
line = separate_row_element(lengths, lines[31915])
print(line)
print(len(line))

['DE10', ' 1', '2020', '28.01.2020', '28.01.2020', '          ', '4500020444', 'RE ', 'E9', 'ESA', ' 19,000', '                792,00-', '                       ', 'EUR  ', '     ', '5521200 ', '5000000   ', 'IT01069540332       ', 'Escar S.r.l.                       ', 'IT ', 'Italien        ', 'Via Rossetti 4                     ', 'Cortemaggiore (PC)                 ', '29016     ', '26.02.2020', '            ', 'Reverse Charge 19 % (§13b Ste ', '         150,48-', '                942,48-', '9105890   ', '         150,48-', '                              ']
32


In [94]:
line = lines[31915]
print(line)
print(len(line))

|DE10| 1|2020|28.01.2020|28.01.2020|          |4500020444|RE |E9|ESA| 19,000|                792,00-|                       |EUR  |     |5521200 |5000000   |IT01069540332       |Escar S.r.l.                       |IT |Italien        |Via Rossetti 4                     |Cortemaggiore (PC)                 |29016     |26.02.2020|            |Reverse Charge 19 % (§13b Ste |         150,48-|                942,48-|9105890   |         150,48-|                              |

473


In [93]:
line = lines[31916]
print(line)
print(len(line))

|DE10| 1|2020|29.01.2020|29.01.2020|          |300009728 |NA |Z4|MWS|  0,000|                435,43-|              3.324,00-|EUR  |CNY  |5521100 |2099840   |                    |Hangzhou  ############ |CN |China          | No.11 Ave No.1,                   |Hangzhou                           |310018    |29.07.2020| 7,63380    |Ausgangs-Dienstleistung 0% (D |           0,00 |                435,43-|9113574   |           0,00 |                              |

461


In [97]:
df = generate_df(new_lines, col, start, save_index)
df.head()

wrong length error: should be: 473,  current: 461 index: 31916
wrong length error: should be: 473,  current: 461 index: 93713
wrong length error: should be: 473,  current: 461 index: 369760
wrong length error: should be: 473,  current: 461 index: 369761
wrong length error: should be: 473,  current: 461 index: 369762
wrong length error: should be: 473,  current: 461 index: 369763
wrong length error: should be: 473,  current: 461 index: 444496
for Ausgangssteuer, there are totoal 413741 rows.


,BuKr,M,Jahr,Buch.dat.,Belegdatum,Meldedatum,Belegnr,Art,St,Vor,...,Ort,Postleitz.,Ausgleich,UmrechKurs,Beschreibung,abzuf. Ausg. St.,Bruttobetrag,Debitor,Ausgangssteuer,Ausgangssteuer in Fremdw.
0,DE10,1,2020,01.01.2020,01.01.2020,,1000007940,AB,A9,MWS,...,Colombo,05,01.01.2020,,Ausgangssteuer Ausland steuer,"0,00","0,01",910373337,"0,00",
1,DE10,1,2020,01.01.2020,01.01.2020,,1000018044,AB,A9,MWS,...,Dhaka,1207,01.01.2020,,Ausgangssteuer Ausland steuer,"0,00","0,01",910451709,"0,00",
2,DE10,1,2020,01.01.2020,01.01.2020,,1000018045,AB,A9,MWS,...,Dhaka,1207,23.01.2020,,Ausgangssteuer Ausland steuer,"0,00","0,99-",910451709,"0,00",
3,DE10,1,2020,01.01.2020,01.01.2020,,1200000012,YC,AZ,MWS,...,,,,,Kein Steuervorgang (SD),"0,00","89.248,77-",,"0,00",
4,DE10,1,2020,01.01.2020,01.01.2020,,1200000014,YC,AZ,MWS,...,Winnenden,71364,17.02.2020,,Kein Steuervorgang (SD),"0,00","5.017,91-",920377020,"0,00",


**now run them together**

remember there are two types, i.e., Augangssteuer & Eingangssteuer

In [27]:
# path of .txt
file_path = '/Users/rampageyao/Desktop/KPMG Work/KPMG/Tax_Cleaning/AKW 2020.TXT' 

# arbitrary symbol from desired classes
key_1 = 'BuKr' 

# options: 'Ausgangssteuer in Fremdw.     ' or 'Eingangssteuer in Fremdw.     '
key_2 = 'Ausgangssteuer in Fremdw.     ' 

# arbitrary symbol from desired data value
key_3 = 'DE10' 

# options: 'csv' or 'excel'
Type = 'csv'

In [28]:
save_index = store_index(file_path, key_1, key_2, key_3)
new_lines, lengths, start, col = return_new_lines(file_path, key_1, key_2, find_length=True)
lines = read_lines(file_path)
df = generate_df(new_lines, col, start, save_index, lines)
export_data(Type, df, key_2)

wrong length error: should be: 473,  current: 461 row index: 31916
wrong length error: should be: 473,  current: 461 row index: 93713
wrong length error: should be: 473,  current: 461 row index: 369760
wrong length error: should be: 473,  current: 461 row index: 369761
wrong length error: should be: 473,  current: 461 row index: 369762
wrong length error: should be: 473,  current: 461 row index: 369763
wrong length error: should be: 473,  current: 461 row index: 444496
for Ausgangssteuer, there are totoal 413741 rows.




In [29]:
# path of .txt
file_path = '/Users/rampageyao/Desktop/KPMG Work/KPMG/Tax_Cleaning/AKW 2020.TXT' 

# arbitrary symbol from desired classes
key_1 = 'BuKr' 

# options: 'Ausgangssteuer in Fremdw.     ' or 'Eingangssteuer in Fremdw.     '
key_2 = 'Eingangssteuer in Fremdw.     ' 

# arbitrary symbol from desired data value
key_3 = 'DE10' 

# options: 'csv' or 'excel'
Type = 'csv'

In [30]:
save_index = store_index(file_path, key_1, key_2, key_3)
new_lines, lengths, start, col = return_new_lines(file_path, key_1, key_2, find_length=True)
lines = read_lines(file_path)
df = generate_df(new_lines, col, start, save_index, lines)
export_data(Type, df, key_2)

wrong length error: should be: 473,  current: 454 row index: 546141
wrong length error: should be: 473,  current: 454 row index: 546478
wrong length error: should be: 473,  current: 461 row index: 611202
wrong length error: should be: 473,  current: 454 row index: 617188
wrong length error: should be: 473,  current: 461 row index: 640120
wrong length error: should be: 473,  current: 461 row index: 716074
wrong length error: should be: 473,  current: 461 row index: 724497
wrong length error: should be: 473,  current: 461 row index: 728511
wrong length error: should be: 473,  current: 461 row index: 728513
wrong length error: should be: 473,  current: 461 row index: 728514
wrong length error: should be: 473,  current: 461 row index: 728515
wrong length error: should be: 473,  current: 461 row index: 728516
wrong length error: should be: 473,  current: 461 row index: 747088
wrong length error: should be: 473,  current: 461 row index: 755966
wrong length error: should be: 473,  current: 46